In [1]:
import torch
from frontend import MFCCTransform
from utils import get_device
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN
from models import WavLM_Base_ECAPA_TDNN

MFCCS = 80
EMBEDDING_SIZE = 192
MODEL = "WavLM-Base-joint_ECAPA-TDNN_Random-Triplet-Mining_Genuine"

device = get_device()
#model = ECAPA_TDNN(input_size=MFCCS, lin_neurons=EMBEDDING_SIZE, device=device)
frozen = False
model = WavLM_Base_ECAPA_TDNN(frozen=frozen, device=device)
model.to(device)

state = torch.load(
    f'../models/{MODEL}_best_model_state.pth')
model.load_state_dict(state)
model.eval()

/home/patrick/speaker_recognition_light/.venv/lib/python3.10/site-packages/s3prl/upstream/byol_s/byol_a/common.py:20: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")
/home/patrick/speaker_recognition_light/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/patrick/speaker_recognition_light/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return 

WavLM_Base_ECAPA_TDNN(
  (frontend): UpstreamExpert(
    (model): WavLM(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): Fp32GroupNorm(512, 512, eps=1e-05, affine=True)
            (3): GELU(approximate='none')
          )
          (1-4): 4 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): GELU(approximate='none')
          )
          (5-6): 2 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): GELU(approximate='none')
          )
        )
      )
      (post_extract_proj): Linear(in_features=512, out_features=768, bias=True)
      (dropout_input): Dropout(p=0.1, inplace=Fa

In [2]:
from utils import load_deepfake_dataset
from torch.utils.data import DataLoader
from dataloader import ValidationDataset, collate_valid_fn

SAMPLE_RATE = 16000
DOWNSAMPLING_VALID = -1
BATCH_SIZE = 16

train_labels, dev_labels, test_labels = load_deepfake_dataset()

#frontend = MFCCTransform(
#    number_output_parameters=MFCCS, sample_rate=SAMPLE_RATE)
frontend = lambda x: x

validation_dataset = ValidationDataset(
    train_labels, frontend=frontend, downsampling=DOWNSAMPLING_VALID)
validation_dataset.data_list = validation_dataset.data_list[validation_dataset.data_list["is_genuine"] == 1]
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                    drop_last=True, num_workers=4, pin_memory=True, collate_fn=collate_valid_fn)

!!!!!!Unknown: labels_BSI_train_genuine.txt
number of deepfakes: 688432
number of genuine: 8883


In [3]:
from utils import ModelValidator

validator = ModelValidator(validation_dataloader, device)

In [4]:
validator.validate_model(model, deepfake_eer=False, mlflow_logging=False)

Generating embeddings:   0%|          | 0/555 [00:00<?, ?it/s]/home/patrick/speaker_recognition_light/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/patrick/speaker_recognition_light/.venv/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Generating embeddings: 100%|██████████| 555/555 [05:38<00:00,  1.64it/s]
Computing pairwise scores: 39422760it [18:44, 35042.90it/s]


(0.8557785147430912,
 0.8886529,
 {'TP': 0, 'TN': 39366846, 'FP': 0, 'FN': 55914},
 -1,
 -1,
 {})